In [1]:
import numpy as np
import pandas as pd
import scipy as scipy

import time

import matplotlib.pyplot as plt

In [2]:
from utils import gradient_descent, nesterov_descent, power_iteration

In [3]:
dim = 1000

B = np.random.randn(dim, dim)
eig_vals, U = np.linalg.eigh(B @ B.T)

true_eig_vals = np.logspace(-2, 2, dim, base=10)

A = U @ np.diag(true_eig_vals) @ U.T

In [4]:
maxiter = 100_000

# Finding Maximum Eigenvalue

In [5]:
%%time
max_eigval, max_eigvec = scipy.sparse.linalg.eigsh(A, k=1, which='LM', maxiter=maxiter, tol=1e-8)
max_eigval

CPU times: user 71.9 ms, sys: 19.3 ms, total: 91.1 ms
Wall time: 119 ms


array([100.])

In [6]:
%%time
x = np.random.normal(0, 1, dim)

max_eigval, max_eigvec = gradient_descent(A, x.copy(), lr=100, which='max', maxiter=1000, tol=1e-10, 
                                          save_history=False)
max_eigval

CPU times: user 140 ms, sys: 2.35 ms, total: 142 ms
Wall time: 150 ms


np.float64(99.99999999470775)

# Finding Minimum Eigenvalue

In [7]:
%%time
min_eigval, min_eigvec = scipy.sparse.linalg.eigsh(A, k=1, which='SM', maxiter=maxiter, tol=1e-8)
min_eigval

CPU times: user 9.63 s, sys: 51 ms, total: 9.68 s
Wall time: 9.71 s


array([0.01])

Gradient descent is needed to find the maximum eigenvalue. This is used to find the optimum learning rate for Nesterov descent.

In [8]:
%%time
x = np.random.normal(0, 1, dim)

max_eigval, _ = gradient_descent(A, x.copy(), lr=100, which='max', maxiter=1000, tol=1e-10, 
                                 save_history=False)

lr = 1 / (2.02 * max_eigval)
out = nesterov_descent(A, x.copy(), lr=lr, which='min', maxiter=maxiter, tol=1e-10, save_history=True)
    
min_eigval, _, min_gd_history = out
min_eigval

CPU times: user 611 ms, sys: 4 ms, total: 615 ms
Wall time: 617 ms


np.float64(0.010020944189180155)

# Finding the top 3 eigenvalues

We use the subspace projection method

In [9]:
working_subspace = []

for i in range(3):
    if len(working_subspace) == 0:
        subspace = None
    else:
        subspace = np.array(working_subspace).T
        
    out = gradient_descent(A, x.copy(), lr=100, which='max', maxiter=1000, tol=1e-10, save_history=False,
                           subspace=subspace)
    max_eigval, max_eigvec = out
    print(max_eigval)
    working_subspace.append(max_eigvec)

99.99999999123207
99.08228096490572
98.17298409047547


In [10]:
true_eig_vals[-3:]

array([ 98.17298406,  99.08228099, 100.        ])

# Finding smalles 3 eigenvalues

In [11]:
max_eigval, _ = gradient_descent(A, x.copy(), lr=100, which='max', maxiter=1000, tol=1e-10, save_history=False)

lr = 1 / (2.02 * max_eigval)

In [12]:
maxiter = 50_000

In [13]:
working_subspace = []

for i in range(3):
    if len(working_subspace) == 0:
        subspace = None
    else:
        subspace = np.array(working_subspace).T
        
    out = nesterov_descent(A, x.copy(), lr=lr, which='min', maxiter=maxiter, tol=1e-10, save_history=False,
                           subspace=subspace)
    min_eigval, min_eigvec = out
    print(min_eigval)
    working_subspace.append(min_eigvec)

0.010020944189180155
0.010093819568543376
0.010186357796717855


In [14]:
true_eig_vals[:3]

array([0.01      , 0.01009262, 0.0101861 ])